### Skip-Gram Word2Vec

In [1]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)

In [2]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

In [3]:
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [4]:
vocab = set(test_sentence)

In [5]:
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [6]:
word_to_ix

{'new': 0,
 'so': 1,
 'lusty': 2,
 'Shall': 3,
 'Proving': 4,
 'thine': 5,
 'worth': 6,
 'shall': 7,
 'shame,': 8,
 'were': 9,
 'lies,': 10,
 'mine': 11,
 'by': 12,
 'cold.': 13,
 'held:': 14,
 'and': 15,
 'sum': 16,
 'praise': 17,
 'within': 18,
 'forty': 19,
 'to': 20,
 'on': 21,
 'in': 22,
 'winters': 23,
 'blood': 24,
 'thriftless': 25,
 'old': 26,
 "feel'st": 27,
 'days;': 28,
 'brow,': 29,
 'more': 30,
 "excuse,'": 31,
 "beauty's": 32,
 'my': 33,
 'where': 34,
 'warm': 35,
 'thy': 36,
 'being': 37,
 'see': 38,
 'Thy': 39,
 "totter'd": 40,
 'Will': 41,
 'own': 42,
 'eyes,': 43,
 'gazed': 44,
 'asked,': 45,
 'How': 46,
 'make': 47,
 'sunken': 48,
 'dig': 49,
 'say,': 50,
 'when': 51,
 'beauty': 52,
 'child': 53,
 'besiege': 54,
 'much': 55,
 'answer': 56,
 'Were': 57,
 'his': 58,
 'all-eating': 59,
 'field,': 60,
 'the': 61,
 'praise.': 62,
 'art': 63,
 "deserv'd": 64,
 'a': 65,
 'succession': 66,
 'trenches': 67,
 'small': 68,
 "youth's": 69,
 'it': 70,
 'an': 71,
 'of': 72,
 'now

In [7]:
import torch.nn as nn
import torch

In [8]:
class SkipGramLangModel(nn.Module):
    def __init__(self,vocab_size, embedding_dim, context_size):
        super(SkipGramLangModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size,embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
    
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1,-1))
        out = torch.relu(self.linear1(embeds))
        out = self.linear2(out)
        logs_probs = torch.log_softmax(out, dim = 1)
        return logs_probs

In [9]:
losses = []
loss_func = nn.NLLLoss()
model = SkipGramLangModel(len(vocab),EMBEDDING_DIM,CONTEXT_SIZE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [10]:
for epochs in range(10):
    total_loss = 0
    for context,target in ngrams:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_func(log_probs,torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

[518.4590497016907, 516.049512386322, 513.6575908660889, 511.2817862033844, 508.92097520828247, 506.57483959198, 504.2426676750183, 501.92250752449036, 499.61424946784973, 497.3170645236969]


In [11]:
model.embeddings.weight[word_to_ix["beauty"]]

tensor([-0.5911,  0.6573,  1.3298, -0.4835, -0.3081, -1.0619,  0.6875, -0.2479,
         0.4607, -0.6335], grad_fn=<SelectBackward0>)